In [1]:
from numpy import array
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, KFold

from sklearn.base import clone

import time

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
columns = ['cases_new','cases_active','cases_death']
target = 'cases_death'
# choose a number of time steps
n_steps = 5
n_features = len(columns)-1
state = 11

In [4]:
df_new = pd.read_csv('train_states.csv', index_col=0)
df_new = df_new.sort_values(by = 'date')
df_new

,date,cases_new,state_id,cases_recovered,cases_death,cases_active,checkins,unique_ind,cumul_full,pop,percent_vax,temp
0,2021-02-24,318,2,518,0,6156,2074008,927705,0,3781000,0.00,25.80
15,2021-02-24,1,17,5,0,51,195780,113866,0,110000,0.00,28.40
14,2021-02-24,5,16,1,0,22,69982,28861,0,99600,0.00,26.20
13,2021-02-24,381,15,432,7,2711,3562969,1647182,0,1773700,0.00,29.40
12,2021-02-24,22,12,63,1,297,354767,175197,0,1259300,0.00,25.95
...,...,...,...,...,...,...,...,...,...,...,...,...
4865,2021-12-25,192,3,184,0,1644,1182090,605241,1568408,2185100,0.72,28.90
4864,2021-12-25,416,2,1232,3,5188,3603159,1506124,3036641,3781000,0.80,27.70
4878,2021-12-25,2,16,10,0,62,71513,28947,79555,99600,0.80,27.70
4870,2021-12-25,164,9,0,4,1899,2093567,915909,1852221,2510300,0.74,27.80


In [5]:
df = df_new[df_new['state_id']==state]
df

,date,cases_new,state_id,cases_recovered,cases_death,cases_active,checkins,unique_ind,cumul_full,pop,percent_vax,temp
11,2021-02-24,581,11,1322,2,10490,4302048,2070162,0,6538000,0.00,28.90
27,2021-02-25,666,11,1604,9,9551,4385034,2115153,0,6538000,0.00,28.25
43,2021-02-26,633,11,1149,4,9034,4655887,2209354,0,6538000,0.00,28.95
59,2021-02-27,862,11,1132,2,8763,5315892,2313602,0,6538000,0.00,29.20
75,2021-02-28,914,11,943,3,8731,5101079,2227024,1,6538000,0.00,28.15
...,...,...,...,...,...,...,...,...,...,...,...,...
4811,2021-12-21,736,11,1172,9,14104,6951905,3221224,4755638,6538000,0.73,29.05
4827,2021-12-22,945,11,1414,4,13635,7126497,3269055,4756874,6538000,0.73,27.25
4843,2021-12-23,951,11,1428,6,13154,7195693,3299399,4758165,6538000,0.73,27.70
4859,2021-12-24,848,11,1263,3,12739,7792725,3467531,4759718,6538000,0.73,28.75


In [6]:
X = df[columns]
X

,cases_new,cases_active,cases_death
11,581,10490,2
27,666,9551,9
43,633,9034,4
59,862,8763,2
75,914,8731,3
...,...,...,...
4811,736,14104,9
4827,945,13635,4
4843,951,13154,6
4859,848,12739,3


In [7]:
# normalize encoder
X_scaler = MinMaxScaler()
X_scaler.fit(X)

y_scaler = MinMaxScaler()
y_scaler.fit(array(df[target]).reshape(-1, 1))

X = X_scaler.transform(X)
X = pd.DataFrame(X, columns = columns)
X

,cases_new,cases_active,cases_death
0,0.042226,0.067006,0.005571
1,0.052140,0.056533,0.025070
2,0.048291,0.050767,0.011142
3,0.075003,0.047745,0.005571
4,0.081068,0.047388,0.008357
...,...,...,...
300,0.060306,0.107312,0.025070
301,0.084684,0.102081,0.011142
302,0.085384,0.096717,0.016713
303,0.073370,0.092088,0.008357


In [8]:
for i in range(n_steps):
    shifted_X = X.shift(i+1)
    X['cn_'+str(i+1)] = shifted_X['cases_new']
    X['ca_'+str(i+1)] = shifted_X['cases_active']
    
X = X.dropna()

In [9]:
y = X[target]

In [11]:
cols = []
features = ['cn', 'ca']

for i in range(n_steps):
    for f in features:
        cols.append(f+"_"+str(i+1))
cols

['cn_1',
 'ca_1',
 'cn_2',
 'ca_2',
 'cn_3',
 'ca_3',
 'cn_4',
 'ca_4',
 'cn_5',
 'ca_5']

In [12]:
X = X[cols]
X

,cn_1,ca_1,cn_2,ca_2,cn_3,ca_3,cn_4,ca_4,cn_5,ca_5
5,0.081068,0.047388,0.075003,0.047745,0.048291,0.050767,0.052140,0.056533,0.042226,0.067006
6,0.027295,0.041198,0.081068,0.047388,0.075003,0.047745,0.048291,0.050767,0.052140,0.056533
7,0.052840,0.038555,0.027295,0.041198,0.081068,0.047388,0.075003,0.047745,0.048291,0.050767
8,0.049108,0.037919,0.052840,0.038555,0.027295,0.041198,0.081068,0.047388,0.075003,0.047745
9,0.047941,0.036938,0.049108,0.037919,0.052840,0.038555,0.027295,0.041198,0.081068,0.047388
...,...,...,...,...,...,...,...,...,...,...
300,0.018780,0.112241,0.081652,0.117862,0.104398,0.120505,0.142541,0.123171,0.136358,0.125703
301,0.060306,0.107312,0.018780,0.112241,0.081652,0.117862,0.104398,0.120505,0.142541,0.123171
302,0.084684,0.102081,0.060306,0.107312,0.018780,0.112241,0.081652,0.117862,0.104398,0.120505
303,0.085384,0.096717,0.084684,0.102081,0.060306,0.107312,0.018780,0.112241,0.081652,0.117862


In [15]:
X = np.array(X).reshape((X.shape[0], n_steps, n_features))
X

array([[[0.08106847, 0.04738803],
        [0.07500292, 0.04774491],
        [0.04829115, 0.05076731],
        [0.05214044, 0.05653328],
        [0.04222559, 0.06700571]],

       [[0.027295  , 0.04119825],
        [0.08106847, 0.04738803],
        [0.07500292, 0.04774491],
        [0.04829115, 0.05076731],
        [0.05214044, 0.05653328]],

       [[0.05284031, 0.03855505],
        [0.027295  , 0.04119825],
        [0.08106847, 0.04738803],
        [0.07500292, 0.04774491],
        [0.04829115, 0.05076731]],

       ...,

       [[0.08468447, 0.1020811 ],
        [0.06030561, 0.10731174],
        [0.01877989, 0.11224126],
        [0.0816517 , 0.11786224],
        [0.10439753, 0.12050544]],

       [[0.08538435, 0.09671663],
        [0.08468447, 0.1020811 ],
        [0.06030561, 0.10731174],
        [0.01877989, 0.11224126],
        [0.0816517 , 0.11786224]],

       [[0.07336988, 0.09208824],
        [0.08538435, 0.09671663],
        [0.08468447, 0.1020811 ],
        [0.06030561, 0.10

In [16]:
y = array(y).reshape(-1, 1)
y

array([[0.        ],
       [0.00835655],
       [0.00278552],
       [0.00835655],
       [0.01392758],
       [0.00835655],
       [0.00557103],
       [0.        ],
       [0.01114206],
       [0.00557103],
       [0.01114206],
       [0.00278552],
       [0.00278552],
       [0.00278552],
       [0.00278552],
       [0.00278552],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.00557103],
       [0.00278552],
       [0.        ],
       [0.        ],
       [0.00557103],
       [0.        ],
       [0.00278552],
       [0.        ],
       [0.00278552],
       [0.00278552],
       [0.00278552],
       [0.01114206],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.01114206],
       [0.00278552],
       [0.        ],
       [0.00557103],
       [0.        ],
       [0.00835655],
       [0.01114206],
       [0.00278552],
       [0.00835655],
       [0.00278552],
       [0.00835655],
       [0.00278552],
       [0.005

In [22]:
# define model
def create_model():
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [23]:
print('length X: ', len(X))
print('length y: ', len(y))

length X:  300
length y:  300


In [24]:
models = []
scores = []
#tscv = TimeSeriesSplit(n_splits=10)
kf = KFold(n_splits=10)
t0 = time.time()

for train_index, test_index in kf.split(X):
    model_new = create_model()
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    t1 = time.time()
    # fit model
    model_new.fit(X_train, y_train, epochs=len(X), verbose=0)
    t_total = time.time()-t1
    print("time-taken: ", int(t_total/60), "mins", int(t_total%60), "seconds", int(t_total*1000%1000), "miliseconds")

    
    y_test_ = y_scaler.inverse_transform(y_test)
    y_pred_ = y_scaler.inverse_transform(model_new.predict(X_test))
    result = mean_squared_error(y_test_, y_pred_)**0.5

    models.append(model_new)
    scores.append(result)

time-taken:  0 mins 8 seconds 49 miliseconds
time-taken:  0 mins 9 seconds 851 miliseconds
time-taken:  0 mins 8 seconds 250 miliseconds
time-taken:  0 mins 8 seconds 904 miliseconds
time-taken:  0 mins 8 seconds 521 miliseconds
time-taken:  0 mins 10 seconds 51 miliseconds
time-taken:  0 mins 9 seconds 333 miliseconds
time-taken:  0 mins 6 seconds 774 miliseconds
time-taken:  0 mins 6 seconds 871 miliseconds
time-taken:  0 mins 6 seconds 764 miliseconds


In [25]:
t_total = time.time()-t0
print("time-taken: ", int(t_total/60), "mins", int(t_total%60), "seconds", int(t_total*1000%1000), "miliseconds")

time-taken:  1 mins 29 seconds 528 miliseconds


In [26]:
scores

[1.4269279465815576,
 8.674283371931612,
 6.374242216716757,
 25.851870652479015,
 25.394898597523635,
 36.949912538522156,
 89.62425233999578,
 10.358199426489682,
 6.533393649762324,
 4.257901161541472]